In [ ]:
import argparse
import os
import sys
from os import mkdir
from apex import amp
import shutil



import torch.nn.functional as F

sys.path.append('..')
from config import cfg
from data import make_data_loader, make_data_loader_view
from engine.trainer import do_train
from modeling import build_model
from solver import make_optimizer, WarmupMultiStepLR,build_scheduler
from layers import make_loss

from utils.logger import setup_logger

from torch.utils.tensorboard import SummaryWriter
import torch
from layers.RaySamplePoint import RaySamplePoint
from engine import render
import numpy as np 
torch.cuda.set_device(2)
import matplotlib.pyplot as plt
import cv2
from tqdm.notebook import trange, tqdm
from imageio_ffmpeg import write_frames
%load_ext autoreload
%autoreload 2

In [ ]:
training_folder = '/data/wmy/NR/RF_training/plant'
epoch = 360000
para_file = 'rfnr_model_%d.pth' % epoch

In [ ]:
cfg.merge_from_file(os.path.join(training_folder,'configs.yml'))
cfg.MODEL.BOARDER_WEIGHT = 1e-10
cfg.freeze()

In [ ]:
val_loader, dataset_val = make_data_loader_view(cfg, is_train=False)
train_loader, dataset = make_data_loader(cfg, is_train=True)

In [ ]:
model = build_model(cfg).cuda()
model.load_state_dict(torch.load(os.path.join(training_folder,para_file),map_location='cpu'))
model.eval()
pass

In [ ]:
img, vs, frame_id, T, K, near_far,_ = dataset_val.NHR_dataset.__getitem__(0)
K = K.unsqueeze(0)

for batch in train_loader:
    rays, rgbs, bboxes,  near_fars, frame_ids = batch
    break

In [ ]:
center = torch.mean(vs,dim=0).cpu()
up = -torch.mean(dataset_val.NHR_dataset.Ts[:,0:3,1],dim =0)
up = up / torch.norm(up)
radius = torch.norm(dataset_val.NHR_dataset.Ts[0,0:3,3] - center) * 0.5

center = center + up*radius*0.0


v = torch.tensor([0,0,-1], dtype=torch.float32)
v = v - up.dot(v)*up
v = v / torch.norm(v)

K[:,0,2] = 400
K[:,1,2] = 300

K = K.cuda()


s_pos = center - v * radius + up*radius*0.1

center = center.numpy()
up = up.numpy()
radius = radius.item()
s_pos = s_pos.numpy()

In [ ]:
import math
def rotate(angle):
    #res = np.array([ [math.cos(angle), -math.sin(angle), 0],[ math.sin(angle),math.cos(angle),0],[0,0,1] ])
    res = np.array([ [math.cos(angle), 0, math.sin(-angle)],[0,1,0],[ math.sin(-angle),0, math.cos(angle)]])
    return res

def rodrigues_rotation_matrix(axis, theta):
    axis = np.asarray(axis)
    theta = np.asarray(theta)
    axis = axis/math.sqrt(np.dot(axis, axis))
    a = math.cos(theta/2.0)
    b, c, d = -axis*math.sin(theta/2.0)
    aa, bb, cc, dd = a*a, b*b, c*c, d*d
    bc, ad, ac, ab, bd, cd = b*c, a*d, a*c, a*b, b*d, c*d
    return np.array([[aa+bb-cc-dd, 2*(bc+ad), 2*(bd-ac)],
                     [2*(bc-ad), aa+cc-bb-dd, 2*(cd+ab)],
                     [2*(bd+ac), 2*(cd-ab), aa+dd-bb-cc]])

index = 0

dx = 1
    
if not os.path.exists(os.path.join(training_folder,'vis_%d'%epoch)):
    os.mkdir(os.path.join(training_folder,'vis_%d'%epoch))
if not os.path.exists(os.path.join(training_folder,'vis_alpha_%d'%epoch)):
    os.mkdir(os.path.join(training_folder,'vis_alpha_%d'%epoch))
if not os.path.exists(os.path.join(training_folder,'vis_depth_%d'%epoch)):
    os.mkdir(os.path.join(training_folder,'vis_depth_%d'%epoch))
    
sKs = []
sTs = []
frames_id = []
    
for i in range(720):

    
    angle = 3.1415926*2*i/360.0
    #angle= 0 
    
    pos = s_pos - center
    pos = rodrigues_rotation_matrix(up,-angle).dot(pos) 
    pos = pos + center
    
    print('pos:',pos)
    
    
    lookat = center - pos
    lookat = lookat/np.linalg.norm(lookat)
    
    xaxis = np.cross(lookat, up)
    xaxis = xaxis / np.linalg.norm(xaxis)
    
    yaxis = -np.cross(xaxis,lookat)
    yaxis = yaxis/np.linalg.norm(yaxis)
    
    nR = np.array([xaxis,yaxis,lookat, pos]).T
    nR = np.concatenate([nR,np.array([[0,0,0,1]])])
    
    sTs.append(nR)
    sKs.append(K[0].cpu().numpy())
    frames_id.append(index)
    
    with torch.no_grad():
        stage2, stage1 = render(model, K[0],torch.Tensor(nR).cuda(),(600,800), bboxes = bboxes[0][0].cuda())
    


    torch.cuda.empty_cache()
    print(i,'/360')
    
    



In [ ]:
import math
import time
def rotate(angle):
    #res = np.array([ [math.cos(angle), -math.sin(angle), 0],[ math.sin(angle),math.cos(angle),0],[0,0,1] ])
    res = np.array([ [math.cos(angle), 0, math.sin(-angle)],[0,1,0],[ math.sin(-angle),0, math.cos(angle)]])
    return res

def rodrigues_rotation_matrix(axis, theta):
    axis = np.asarray(axis)
    theta = np.asarray(theta)
    axis = axis/math.sqrt(np.dot(axis, axis))
    a = math.cos(theta/2.0)
    b, c, d = -axis*math.sin(theta/2.0)
    aa, bb, cc, dd = a*a, b*b, c*c, d*d
    bc, ad, ac, ab, bd, cd = b*c, a*d, a*c, a*b, b*d, c*d
    return np.array([[aa+bb-cc-dd, 2*(bc+ad), 2*(bd-ac)],
                     [2*(bc-ad), aa+cc-bb-dd, 2*(cd+ab)],
                     [2*(bd+ac), 2*(cd-ab), aa+dd-bb-cc]])

index = 0

dx = 1
    
if not os.path.exists(os.path.join(training_folder,'vis_%d'%epoch)):
    os.mkdir(os.path.join(training_folder,'vis_%d'%epoch))
if not os.path.exists(os.path.join(training_folder,'vis_alpha_%d'%epoch)):
    os.mkdir(os.path.join(training_folder,'vis_alpha_%d'%epoch))
if not os.path.exists(os.path.join(training_folder,'vis_depth_%d'%epoch)):
    os.mkdir(os.path.join(training_folder,'vis_depth_%d'%epoch))
    
    
sKs = []
sTs = []
frames_id = []


latitde = 2

longitude_step = lambda x: int((1.0 - float(abs(x))/latitde)*120 + 60)



    
for j in range(0,latitde):

    print('latitde:',j)

    s_pos = (center - radius*v.numpy()  + up*radius*0.01*j)



    for i in range(longitude_step(j)):

        beg = time.time()
        angle_step = 180.0/ longitude_step(j)

        angle = 3.1415926*4*i*angle_step/360.0

        #angle= 0 

        pos = s_pos - center
        pos = rodrigues_rotation_matrix(up,-angle).dot(pos) 

        pos = pos + center

        print('pos:',pos)


        lookat = center - pos
        lookat = lookat/np.linalg.norm(lookat)

        xaxis = np.cross(lookat, up)
        xaxis = xaxis / np.linalg.norm(xaxis)

        yaxis = -np.cross(xaxis,lookat)
        yaxis = yaxis/np.linalg.norm(yaxis)

        nR = np.array([xaxis,yaxis,lookat, pos]).T
        nR = np.concatenate([nR,np.array([[0,0,0,1]])])

        sTs.append(nR)
        sKs.append(K[0].cpu().numpy())
        frames_id.append('img_%04d_%d_%d.jpg'%(index,j,i))

        with torch.no_grad():
            stage2, stage1 = render(model, K[0],torch.Tensor(nR).cuda(),(600,800),  bboxes = bboxes[0][0].cuda())  # ROI=(30,280,570,220),
            
            
        img = stage2[0].detach().cpu().numpy()
        depth = stage2[1].detach().cpu().numpy()
        alpha = stage2[2].detach().cpu().numpy()
        
        print('render time:', time.time()-beg)



        cv2.imwrite(os.path.join(training_folder,'vis_%d/img_%04d_%d_%d.jpg'%(epoch,index,j,i)),img)

        cv2.imwrite(os.path.join(training_folder,'vis_alpha_%d/img_%04d_%d_%d.jpg'%(epoch,index,j,i)  ),alpha)
        cv2.imwrite(os.path.join(training_folder,'vis_depth_%d/img_%04d_%d_%d.jpg'%(epoch,index,j,i)  ),depth)




        torch.cuda.empty_cache()
        print(i)
        break
    break



with open(os.path.join(training_folder,'Intrinsic_%d.inf'%epoch), 'w') as f:
    for i,camk in enumerate(sKs):
        f.write('%d\n'%i)
        f.write('%f %f %f\n %f %f %f\n %f %f %f\n' % tuple(camk.reshape(9).tolist()))
        f.write('\n')


with open(os.path.join(training_folder,'CamPose_%d.inf' %epoch), 'w') as f:
    for i,camT in enumerate(sTs):
        A = camT[0:3,:]
        tmp = np.concatenate( [A[0:3,2].T, A[0:3,0].T,A[0:3,1].T,A[0:3,3].T])
        f.write('%f %f %f %f %f %f %f %f %f %f %f %f\n' % tuple(tmp.tolist()))

with open(os.path.join(training_folder,'frames_%d.inf' %epoch), 'w') as f:
    for i,ids in enumerate(frames_id):
        f.write('%s\n' % ids)

In [ ]:
alpha.shape

In [ ]:
img =  img.reshape(600,800, 3)

In [ ]:
plt.figure(figsize=(16,10))
plt.imshow(img)

In [ ]:
K

In [ ]:
#center = np.array([0.0174 ,  -0.772931, -4.5986,1 ])
#up = np.array([0, 1, 0,0])

#center = Ts.dot(center)[0:3]
#up = Ts.dot(up)[0:3]

import math
def rotate(angle):
    #res = np.array([ [math.cos(angle), -math.sin(angle), 0],[ math.sin(angle),math.cos(angle),0],[0,0,1] ])
    res = np.array([ [math.cos(angle), 0, math.sin(-angle)],[0,1,0],[ math.sin(-angle),0, math.cos(angle)]])
    return res

def rodrigues_rotation_matrix(axis, theta):
    axis = np.asarray(axis)
    theta = np.asarray(theta)
    axis = axis/math.sqrt(np.dot(axis, axis))
    a = math.cos(theta/2.0)
    b, c, d = -axis*math.sin(theta/2.0)
    aa, bb, cc, dd = a*a, b*b, c*c, d*d
    bc, ad, ac, ab, bd, cd = b*c, a*d, a*c, a*b, b*d, c*d
    return np.array([[aa+bb-cc-dd, 2*(bc+ad), 2*(bd-ac)],
                     [2*(bc-ad), aa+cc-bb-dd, 2*(cd+ab)],
                     [2*(bd+ac), 2*(cd-ab), aa+dd-bb-cc]])

index = 0

dx = 1
    
if not os.path.exists(os.path.join(training_folder,'vis_%d'%epoch)):
    os.mkdir(os.path.join(training_folder,'vis_%d'%epoch))
#if not os.path.exists(os.path.join(model_path,'vis_mask_%d'%epoch)):
#    os.mkdir(os.path.join(model_path,'vis_mask_%d'%epoch))
#if not os.path.exists(os.path.join(model_path,'vis_depth_%d'%epoch)):
#    os.mkdir(os.path.join(model_path,'vis_depth_%d'%epoch))
    
#if not os.path.exists(os.path.join(model_path,'vis_compos_%d'%epoch)):
#    os.mkdir(os.path.join(model_path,'vis_compos_%d'%epoch))
    
sKs = []
sTs = []
frames_id = []

out_img_size = (800,600)

latitde = 1

longitude_step = lambda x: int((1.0 - float(abs(x))/latitde)*45 + 45)

writer_raw_rgb = write_frames(os.path.join(os.path.join(training_folder,'vis_%d'%epoch),'vis_frame%04d.mp4'%index), out_img_size,fps=15, macro_block_size =8,quality=6)  # size is (width, height)
writer_raw_mask = write_frames(os.path.join(os.path.join(training_folder,'vis_%d'%epoch),'vis_frame%04d_mask.mp4'%index), out_img_size,fps=15, macro_block_size =8,quality=6)  # size is (width, height)
writer_raw_depth = write_frames(os.path.join(os.path.join(training_folder,'vis_%d'%epoch),'vis_frame%04d_depth.mp4'%index), out_img_size,fps=15, macro_block_size =8,quality=6)  # size is (width, height)

writer_raw_rgb.send(None)
writer_raw_mask.send(None)
writer_raw_depth.send(None)
   

    
for j in range(1):




    for i in trange(longitude_step(j)):


        angle_step = 180.0/ longitude_step(j)

        angle = 3.1415926*4*i*angle_step/360.0

        #angle= 0 
        s_pos = (center - radius*v.numpy()  + up*radius*0.4*abs(math.sin(3.1415926*2*i/longitude_step(j))) + up*radius*0.1)
        pos = s_pos - center
        pos = rodrigues_rotation_matrix(up,-angle).dot(pos) 

        pos = pos + center

        #print('pos:',pos)


        lookat = center - pos
        lookat = lookat/np.linalg.norm(lookat)

        xaxis = np.cross(lookat, up)
        xaxis = xaxis / np.linalg.norm(xaxis)

        yaxis = -np.cross(xaxis,lookat)
        yaxis = yaxis/np.linalg.norm(yaxis)

        nR = np.array([xaxis,yaxis,lookat, pos]).T
        nR = np.concatenate([nR,np.array([[0,0,0,1]])])

        sTs.append(nR)
        sKs.append(K[0].cpu().numpy())
        frames_id.append('img_%04d_%d_%d.jpg'%(index,i,j))


    
        with torch.no_grad():
            stage2, stage1 = render(model, K[0],torch.Tensor(nR).cuda(),(out_img_size[1],out_img_size[0]),  bboxes = bboxes[0][0].cuda())  # ROI=(30,280,570,220),

        img = stage2[0].detach().cpu().numpy()*255.0
        depth = stage2[1].detach().cpu()
        alpha = stage2[2].detach().cpu()
        
        #print('render time:', time.time()-beg)

        print(alpha.shape)

        cv2.imwrite(os.path.join(training_folder,'vis_%d/img_%04d_%d_%d.jpg'%(epoch,index,j,i)),img)

        #cv2.imwrite(os.path.join(training_folder,'vis_alpha_%d/img_%04d_%d_%d.jpg'%(epoch,index,j,i)  ),alpha)
        #cv2.imwrite(os.path.join(training_folder,'vis_depth_%d/img_%04d_%d_%d.jpg'%(epoch,index,j,i)  ),depth)

        mask = alpha.unsqueeze(-1).repeat(1,1,3).numpy()*255
        img_depth = depth.unsqueeze(-1).repeat(1,1,3)
        img_depth = img_depth/img_depth.max()
        img_depth = img_depth*255.0
        img_depth = img_depth.numpy()
        writer_raw_rgb.send(img.astype(np.uint8))
        writer_raw_mask.send(mask.astype(np.uint8))
        writer_raw_depth.send(img_depth.astype(np.uint8))



        torch.cuda.empty_cache()
        #print(i,'/360')
        
        
    break
        
writer_raw_rgb.close()
writer_raw_mask.close()
writer_raw_depth.close()
    
    



with open(os.path.join(training_folder,'Intrinsic_%d.inf'%epoch), 'w') as f:
    for i,camk in enumerate(sKs):
        f.write('%d\n'%i)
        f.write('%f %f %f\n %f %f %f\n %f %f %f\n' % tuple(camk.reshape(9).tolist()))
        f.write('\n')


with open(os.path.join(training_folder,'CamPose_%d.inf' %epoch), 'w') as f:
    for i,camT in enumerate(sTs):
        A = camT[0:3,:]
        tmp = np.concatenate( [A[0:3,2].T, A[0:3,0].T,A[0:3,1].T,A[0:3,3].T])
        f.write('%f %f %f %f %f %f %f %f %f %f %f %f\n' % tuple(tmp.tolist()))

with open(os.path.join(training_folder,'frames_%d.inf' %epoch), 'w') as f:
    for i,ids in enumerate(frames_id):
        f.write('%s\n' % ids)

In [ ]:
K = K*1.5
K[0,2,2] = 1.0

In [ ]:
print(K)

In [ ]:
print(model.spacenet)

In [ ]:
torch.save(model.spacenet.state_dict(),'spacenet.pth')